# Day 5: Supply Stacks

The expedition can depart as soon as the final supplies have been unloaded from the ships. Supplies are stored in stacks of marked crates, but because the needed supplies are buried under many other crates, the crates need to be rearranged.

The ship has a giant cargo crane capable of moving crates between stacks. To ensure none of the crates get crushed or fall over, the crane operator will rearrange them in a series of carefully-planned steps. After the crates are rearranged, the desired crates will be at the top of each stack.

The Elves don't want to interrupt the crane operator during this delicate procedure, but they forgot to ask her which crate will end up where, and they want to be ready to unload them as soon as possible so they can embark.

They do, however, have a drawing of the starting stacks of crates and the rearrangement procedure (your puzzle input). For example:

```
    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2
```

In this example, there are three stacks of crates. Stack 1 contains two crates: crate Z is on the bottom, and crate N is on top. Stack 2 contains three crates; from bottom to top, they are crates M, C, and D. Finally, stack 3 contains a single crate, P.

Then, the rearrangement procedure is given. In each step of the procedure, a quantity of crates is moved from one stack to a different stack. In the first step of the above rearrangement procedure, one crate is moved from stack 2 to stack 1, resulting in this configuration:

```
[D]        
[N] [C]    
[Z] [M] [P]
 1   2   3 
```

In the second step, three crates are moved from stack 1 to stack 3. Crates are moved one at a time, so the first crate to be moved (D) ends up below the second and third crates:

```
        [Z]
        [N]
    [C] [D]
    [M] [P]
 1   2   3
```

Then, both crates are moved from stack 2 to stack 1. Again, because crates are moved one at a time, crate C ends up below crate M:

```
        [Z]
        [N]
[M]     [D]
[C]     [P]
 1   2   3
```

Finally, one crate is moved from stack 1 to stack 2:

```
        [Z]
        [N]
        [D]
[C] [M] [P]
 1   2   3
```

The Elves just need to know which crate will end up on top of each stack; in this example, the top crates are C in stack 1, M in stack 2, and Z in stack 3, so you should combine these together and give the Elves the message CMZ.

After the rearrangement procedure completes, what crate ends up on top of each stack?

In [144]:
#@title Imports
import re

In [145]:
#@title Text processing

"""
This function takes raw input lines such as:
    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

and convets it to a 2d list with the first dimension representing rows 1 - 3 
and the second representing the stack of crates. 
We can use the pop() method to get the latest and append() to add a new latest
"""
def import_crates_layout(lines: []) -> list:
    rows = []
    for i, c in enumerate(lines[-1]):
        if c != ' ':
            column = []
            for crate in range(len(lines) -1):
                if lines[crate][i] not in (' ', '[', ']'):
                    column.insert(0, lines[crate][i])
            rows.append(column)
            
    return rows

CRATE_MOVE_REGEX = 'move (?P<quantity>\\d+) from (?P<start>\\d+) to (?P<end>\\d+)'

"""
convert a crate movement instruction from the line format
move 1 from 2 to 1
to a dictionary format:
{'quantity': '1', 'start': '2', 'end': '1'}
"""
def import_crate_instructions(lines: []) -> list:
    instructions = []
    for line in lines:
        matches = re.finditer(CRATE_MOVE_REGEX, line)
        for match in matches:
            instructions.append(match.groupdict())

    return instructions


""" 
This function pulls in the raw input file and converts it to useful a data structure
"""
def import_file_data(filename : str):
    crates_layout_definition = []
    crates_moves_definition = []
    layout_data=True
    
    with open(filename) as f:
        for line in f:
            trimmed_line = line.rstrip('\n')
            if layout_data:
                if trimmed_line == '':
                    layout_data=False
                else:
                    crates_layout_definition.append(trimmed_line)
            else:
                crates_moves_definition.append(trimmed_line)
                
    return import_crates_layout(crates_layout_definition), import_crate_instructions(crates_moves_definition)

"""
This function prints out a given crate layout
"""
def print_crates(crate_columns):
    printable = []

    # Add columns
    column_names = '  1'
    for i in range(1, len(crate_columns)):
        column_names = column_names + f'   {i+1}'
    column_names = f'{column_names}  '
    printable.append(column_names)

    longest_list = len(max(crate_columns, key=len))

    # now loop through and add the new values
    for height in range(longest_list):
        row = ' '
        for i, cc in enumerate(crate_columns):
            if height <len(cc):
                row = f'{row}[{cc[height]}] '
            else:
                row = f'{row}    '
        printable.insert(0, row)

    # for p in printable:
    #     print(p)
        
    return printable
    

In [146]:
#@title Test contains_at_all functions
import unittest

class TestImportFunctions(unittest.TestCase):

    def test_import_crates_layout_case_1(self):
        input = [
            '    [D]    ',
            '[N] [C]    ',
            '[Z] [M] [P]',
            ' 1   2   3 '
        ]
        expected = [
            ['Z', 'N'],
            ['M', 'C', 'D'],
            ['P']
        ]
        self.assertEqual(import_crates_layout(input), expected)

    def test_import_crates_instructions_simple(self):
        input = [
            'move 1 from 2 to 1'
        ]
        expected = [{'quantity': '1', 'start': '2', 'end': '1'}]
        self.assertEqual(import_crate_instructions(input), expected)

    def test_import_crates_instructions_more_realistic(self):
        input = [
            'move 1 from 2 to 1',
            'move 3 from 1 to 3',
            'move 2 from 2 to 1',
            'move 1 from 1 to 2'
        ]
        expected = [
            {'quantity': '1', 'start': '2', 'end': '1'},
            {'quantity': '3', 'start': '1', 'end': '3'},
            {'quantity': '2', 'start': '2', 'end': '1'},
            {'quantity': '1', 'start': '1', 'end': '2'}
        ]
        self.assertEqual(import_crate_instructions(input), expected)

    def test_import_file_data(self):
        crates, moves = import_file_data('test.txt')
        self.assertEqual(crates, [
            ['Z', 'N'],
            ['M', 'C', 'D'],
            ['P']
        ])
        self.assertEqual(moves, [
            {'quantity': '1', 'start': '2', 'end': '1'},
            {'quantity': '3', 'start': '1', 'end': '3'},
            {'quantity': '2', 'start': '2', 'end': '1'},
            {'quantity': '1', 'start': '1', 'end': '2'}
        ])

    def test_print_crates(self):
        input = [
            ['Z', 'N'],
            ['M', 'C', 'D'],
            ['P']
        ]
        expected = [
            '     [D]     ',
            ' [N] [C]     ',
            ' [Z] [M] [P] ',
            '  1   2   3  '
        ]
        actual = print_crates(input)

        for i in range(len(expected)):
            self.assertEqual(actual[i], expected[i], f"Compare of entry {i} failed")
        
        
unittest.main(argv=[''], verbosity=2, exit=False)

test_import_crates_instructions_more_realistic (__main__.TestImportFunctions.test_import_crates_instructions_more_realistic) ... ok
test_import_crates_instructions_simple (__main__.TestImportFunctions.test_import_crates_instructions_simple) ... ok
test_import_crates_layout_case_1 (__main__.TestImportFunctions.test_import_crates_layout_case_1) ... ok
test_import_file_data (__main__.TestImportFunctions.test_import_file_data) ... ok
test_print_crates (__main__.TestImportFunctions.test_print_crates) ... ok
test_process_a_move_9001_case_1 (__main__.TestMove9001Functions.test_process_a_move_9001_case_1) ... ok
test_process_a_move_9001_case_2 (__main__.TestMove9001Functions.test_process_a_move_9001_case_2) ... FAIL
test_process_a_move_9001_case_3 (__main__.TestMove9001Functions.test_process_a_move_9001_case_3) ... FAIL
test_process_a_move_9001_case_4 (__main__.TestMove9001Functions.test_process_a_move_9001_case_4) ... ok
test_process_a_move_case_1 (__main__.TestMoveFunctions.test_process_a_mo

Before pop
Popped D
['D']
Before pop
Popped D
['D']
Before pop
Popped N
['D', 'N']
Before pop
Popped Z
['D', 'N', 'Z']
Before pop
Popped C
['C']
Before pop
Popped M
['C', 'M']
Before pop
Popped C
['C']


In [147]:
#@title Import File data to data structure for processing
crates, moves = import_file_data('input.txt')

In [148]:
print_crates(crates)

['             [J] [Z] [G]             ',
 '             [Z] [T] [S] [P] [R]     ',
 ' [R]         [Q] [V] [B] [G] [J]     ',
 ' [W] [W]     [N] [L] [V] [W] [C]     ',
 ' [F] [Q]     [T] [G] [C] [T] [T] [W] ',
 ' [H] [D] [W] [W] [H] [T] [R] [M] [B] ',
 ' [T] [G] [T] [R] [B] [P] [B] [G] [G] ',
 ' [S] [S] [B] [D] [F] [L] [Z] [N] [L] ',
 '  1   2   3   4   5   6   7   8   9  ']

In [149]:
#@title Process a move

def process_a_move(crates, move):

    start_idx=int(move['start']) - 1
    end_idx=int(move['end']) - 1
    
    for i in range(int(move['quantity'])): 
        to_move = crates[start_idx].pop()
        crates[end_idx].append(to_move)
        
    
    return crates

In [150]:
#@title Test contains_at_all functions
import unittest

class TestMoveFunctions(unittest.TestCase):

    def test_process_a_move_case_1(self):
        input = import_crates_layout([
            '    [D]    ',
            '[N] [C]    ',
            '[Z] [M] [P]',
            ' 1   2   3 '
        ])
        output_expected = import_crates_layout([
            '[D]        ',
            '[N] [C]    ',
            '[Z] [M] [P]',
            ' 1   2   3 '
        ])
        
        self.assertEqual(process_a_move(input, {'quantity': '1', 'start': '2', 'end': '1'}), output_expected)

    def test_process_a_move_case_2(self):
        input = import_crates_layout([
            '[D]        ',
            '[N] [C]    ',
            '[Z] [M] [P]',
            ' 1   2   3 '
        ])
        output_expected = import_crates_layout([
            '        [Z]',
            '        [N]',
            '    [C] [D]',
            '    [M] [P]',
            ' 1   2   3 '
        ])
        
        self.assertEqual(process_a_move(input, {'quantity': '3', 'start': '1', 'end': '3'}), output_expected)

    def test_process_a_move_case_3(self):
        input = import_crates_layout([
            '        [Z]',
            '        [N]',
            '    [C] [D]',
            '    [M] [P]',
            ' 1   2   3 '
        ])
        output_expected = import_crates_layout([
            '        [Z]',
            '        [N]',
            '[M]     [D]',
            '[C]     [P]',
            ' 1   2   3 '
        ])
        
        self.assertEqual(process_a_move(input, {'quantity': '2', 'start': '2', 'end': '1'}), output_expected)

    def test_process_a_move_case_4(self):
        input = import_crates_layout([
            '        [Z]',
            '        [N]',
            '[M]     [D]',
            '[C]     [P]',
            ' 1   2   3 '
        ])
        output_expected = import_crates_layout([
            '        [Z]',
            '        [N]',
            '        [D]',
            '[C] [M] [P]',
            ' 1   2   3 '
        ])
        
        self.assertEqual(process_a_move(input, {'quantity': '1', 'start': '1', 'end': '2'}), output_expected)

unittest.main(argv=[''], verbosity=2, exit=False)

test_import_crates_instructions_more_realistic (__main__.TestImportFunctions.test_import_crates_instructions_more_realistic) ... ok
test_import_crates_instructions_simple (__main__.TestImportFunctions.test_import_crates_instructions_simple) ... ok
test_import_crates_layout_case_1 (__main__.TestImportFunctions.test_import_crates_layout_case_1) ... ok
test_import_file_data (__main__.TestImportFunctions.test_import_file_data) ... ok
test_print_crates (__main__.TestImportFunctions.test_print_crates) ... ok
test_process_a_move_9001_case_1 (__main__.TestMove9001Functions.test_process_a_move_9001_case_1) ... ok
test_process_a_move_9001_case_2 (__main__.TestMove9001Functions.test_process_a_move_9001_case_2) ... FAIL
test_process_a_move_9001_case_3 (__main__.TestMove9001Functions.test_process_a_move_9001_case_3) ... FAIL
test_process_a_move_9001_case_4 (__main__.TestMove9001Functions.test_process_a_move_9001_case_4) ... ok
test_process_a_move_case_1 (__main__.TestMoveFunctions.test_process_a_mo

Before pop
Popped D
['D']
Before pop
Popped D
['D']
Before pop
Popped N
['D', 'N']
Before pop
Popped Z
['D', 'N', 'Z']
Before pop
Popped C
['C']
Before pop
Popped M
['C', 'M']
Before pop
Popped C
['C']


In [151]:
for move in moves:
    crates = process_a_move(crates, move)

print_crates(crates)

['                         [C]         ',
 '                         [N]         ',
 '                         [S]         ',
 '                         [F]         ',
 '                         [W]         ',
 '                         [Q]         ',
 '                         [F]         ',
 '     [R]                 [T]         ',
 '     [J]         [G]     [B]         ',
 '     [M]         [N]     [T]         ',
 '     [R]         [W]     [R]         ',
 '     [Z]         [B]     [G]         ',
 '     [D]         [C]     [W]         ',
 '     [T] [L] [J] [L]     [V]         ',
 '     [W] [T] [Q] [G]     [B]         ',
 '     [S] [T] [L] [P]     [H]     [R] ',
 '     [T] [W] [P] [B] [S] [W]     [G] ',
 ' [Z] [G] [V] [G] [Z] [H] [D] [T] [B] ',
 '  1   2   3   4   5   6   7   8   9  ']

That's the right answer! You are one gold star closer to collecting enough star fruit. [Continue to Part Two]

## Part Two

As you watch the crane operator expertly rearrange the crates, you notice the process isn't following your prediction.

Some mud was covering the writing on the side of the crane, and you quickly wipe it away. T
he crane isn't a CrateMover 9000 - it's a CrateMover 9001.

The CrateMover 9001 is notable for many new and exciting features: air conditioning, 
leather seats, an extra cup holder, and the ability to pick up and move multiple crates at once.

Again considering the example above, the crates begin in the same configuration:

```
    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 
```
Moving a single crate from stack 2 to stack 1 behaves the same as before:

```
[D]        
[N] [C]    
[Z] [M] [P]
 1   2   3 
```

However, the action of moving three crates from stack 1 to stack 3 means that those three moved crates stay in the same order, resulting in this new configuration:

```
        [D]
        [N]
    [C] [Z]
    [M] [P]
 1   2   3
```

Next, as both crates are moved from stack 2 to stack 1, they retain their order as well:

```
        [D]
        [N]
[C]     [Z]
[M]     [P]
 1   2   3
```

Finally, a single crate is still moved from stack 1 to stack 2, but now it's crate C that gets moved:

```
        [D]
        [N]
        [Z]
[M] [C] [P]
 1   2   3
```

In this example, the CrateMover 9001 has put the crates in a totally different order: MCD.

Before the rearrangement process finishes, update your simulation so that the Elves know where they should stand to be ready to unload the final supplies. After the rearrangement procedure completes, what crate ends up on top of each stack?

In [152]:
#@title Process a move

def process_a_move_9001(crates, move):

    start_idx=int(move['start']) - 1
    end_idx=int(move['end']) - 1
    qty = int(move['quantity'])
    start_len = len(crates[start_idx])

    # take all elements from the end of the list and save to temp
    temp = crates[start_idx][start_len - qty:]
    # remove all the taken elements from the start list
    crates[start_idx] = crates[start_idx][:start_len - qty]  
    # add the removed elements we saved to the target list
    crates[end_idx].extend(temp)
        
    return crates

In [153]:
#@title Test contains_at_all functions
import unittest

class TestMove9001Functions(unittest.TestCase):

    def test_process_a_move_9001_case_1(self):
        input = import_crates_layout([
            '    [D]    ',
            '[N] [C]    ',
            '[Z] [M] [P]',
            ' 1   2   3 '
        ])
        output_expected = import_crates_layout([
            '[D]        ',
            '[N] [C]    ',
            '[Z] [M] [P]',
            ' 1   2   3 '
        ])
        
        self.assertEqual(process_a_move_9001(input, {'quantity': '1', 'start': '2', 'end': '1'}), output_expected)

    def test_process_a_move_9001_case_2(self):
        input = import_crates_layout([
            '[D]        ',
            '[N] [C]    ',
            '[Z] [M] [P]',
            ' 1   2   3 '
        ])
        output_expected = import_crates_layout([
            '        [D]',
            '        [N]',
            '    [C] [Z]',
            '    [M] [P]',
            ' 1   2   3 '
        ])
        
        self.assertEqual(process_a_move_9001(input, {'quantity': '3', 'start': '1', 'end': '3'}), output_expected)

    def test_process_a_move_9001_case_3(self):
        input = import_crates_layout([
            '        [D]',
            '        [N]',
            '    [C] [Z]',
            '    [M] [P]',
            ' 1   2   3 '
        ])
        output_expected = import_crates_layout([
            '        [D]',
            '        [N]',
            '[C]     [Z]',
            '[M]     [P]',
            ' 1   2   3 '
        ])
        
        self.assertEqual(process_a_move_9001(input, {'quantity': '2', 'start': '2', 'end': '1'}), output_expected)

    def test_process_a_move_9001_case_4(self):
        input = import_crates_layout([
            '        [D]',
            '        [N]',
            '[C]     [Z]',
            '[M]     [P]',
            ' 1   2   3 '
        ])
        output_expected = import_crates_layout([
            '        [D]',
            '        [N]',
            '        [Z]',
            '[M] [C] [P]',
            ' 1   2   3 '
        ])
        
        self.assertEqual(process_a_move_9001(input, {'quantity': '1', 'start': '1', 'end': '2'}), output_expected)

unittest.main(argv=[''], verbosity=2, exit=False)

test_import_crates_instructions_more_realistic (__main__.TestImportFunctions.test_import_crates_instructions_more_realistic) ... ok
test_import_crates_instructions_simple (__main__.TestImportFunctions.test_import_crates_instructions_simple) ... ok
test_import_crates_layout_case_1 (__main__.TestImportFunctions.test_import_crates_layout_case_1) ... ok
test_import_file_data (__main__.TestImportFunctions.test_import_file_data) ... ok
test_print_crates (__main__.TestImportFunctions.test_print_crates) ... ok
test_process_a_move_9001_case_1 (__main__.TestMove9001Functions.test_process_a_move_9001_case_1) ... ok
test_process_a_move_9001_case_2 (__main__.TestMove9001Functions.test_process_a_move_9001_case_2) ... ok
test_process_a_move_9001_case_3 (__main__.TestMove9001Functions.test_process_a_move_9001_case_3) ... ok
test_process_a_move_9001_case_4 (__main__.TestMove9001Functions.test_process_a_move_9001_case_4) ... ok
test_process_a_move_case_1 (__main__.TestMoveFunctions.test_process_a_move_c

In [161]:
#@title Import Data fresh and recalculate
crates, moves = import_file_data('input.txt')

for move in moves:
    crates = process_a_move_9001(crates, move)

print_crates(crates)

['                         [F]         ',
 '                         [W]         ',
 '                         [S]         ',
 '                         [G]         ',
 '                         [C]         ',
 '                         [T]         ',
 '                         [Q]         ',
 '     [R]                 [F]         ',
 '     [R]         [G]     [T]         ',
 '     [B]         [N]     [B]         ',
 '     [Z]         [W]     [Q]         ',
 '     [H]         [W]     [Z]         ',
 '     [L]         [S]     [R]         ',
 '     [G] [T] [T] [T]     [D]         ',
 '     [W] [J] [D] [B]     [L]         ',
 '     [H] [C] [G] [N]     [W]     [B] ',
 '     [B] [M] [W] [J] [R] [T]     [G] ',
 ' [P] [S] [Z] [L] [T] [G] [V] [P] [V] ',
 '  1   2   3   4   5   6   7   8   9  ']